In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import time


In [13]:
# Inicializar MediaPipe para la detección de manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)


## Con Media Pipe

In [20]:
cap = cv2.VideoCapture(0)

# Verifica si la cámara se ha abierto correctamente
if not cap.isOpened():
    print("Error: No se puede acceder a la cámara.")
    exit()

frame_count = 0  # Contador de cuadros procesados
frame_delay = 10  # Espera de 10 ms entre cuadros (aproximadamente 100 FPS)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el cuadro de la cámara.")
        break

    # Solo procesar cada 2 cuadros (por ejemplo, cada segundo cuadro)
    frame_count += 1
    if frame_count % 50 != 0:
        cv2.imshow('Sign Language Detection', frame)
        continue  # Saltar este cuadro y continuar con el siguiente
    

    # Convertir el cuadro de BGR a RGB para MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar el cuadro con MediaPipe para detectar la mano
    results = hands.process(frame_rgb)

    # Si se detectan manos, extraemos la región de la mano
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            # Obtener las coordenadas del centro de la mano (puedes usar diferentes puntos de los landmarks)
            x_min = int(min([lm.x for lm in landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in landmarks.landmark]) * frame.shape[0])

            # Dibujar un rectángulo alrededor de la mano
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Recortar la región de interés (la mano)
            hand_roi = frame[y_min:y_max, x_min:x_max]

            # Realizar la predicción de YOLO solo en la región de la mano
            if hand_roi.size != 0:
                results_yolo = model(hand_roi, conf=0.7)  # Umbral de confianza de 0.7

                # Obtener los resultados (predicciones) y dibujarlos en el cuadro
                for result in results_yolo[0].boxes.data:
                    x1, y1, x2, y2 = result[:4].int().tolist()
                    conf = result[4].item()
                    class_id = int(result[5].item())
                    class_name = results_yolo[0].names[class_id]

                    # Dibujar la caja delimitadora sobre la imagen de la mano
                    hand_roi = cv2.rectangle(hand_roi, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    hand_roi = cv2.putText(hand_roi, f'{class_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Colocar la región de la mano procesada en la imagen original
            frame[y_min:y_max, x_min:x_max] = hand_roi

    # Mostrar el cuadro con OpenCV
    cv2.imshow('Sign Language Detection', frame)

    # Salir con la tecla 'q'
    if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
        break

# Cerrar la cámara y las ventanas de OpenCV
cap.release()
cv2.destroyAllWindows()


0: 800x608 (no detections), 1451.5ms
Speed: 17.0ms preprocess, 1451.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 608)

0: 736x800 (no detections), 1605.4ms
Speed: 6.9ms preprocess, 1605.4ms inference, 1.0ms postprocess per image at shape (1, 3, 736, 800)

0: 800x800 (no detections), 2018.3ms
Speed: 7.2ms preprocess, 2018.3ms inference, 3.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 (no detections), 2280.8ms
Speed: 18.3ms preprocess, 2280.8ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 608x800 (no detections), 1684.5ms
Speed: 5.9ms preprocess, 1684.5ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


## Sin Media Pipe para el modelo 1

In [2]:
# Cargar el modelo YOLOv8 y optimizar
model = YOLO('best_1.pt')
model.fp16 = True  # Habilitar inferencia en FP16

In [5]:
# Configurar la cámara para menor resolución
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Reducir resolución
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)  # Limitar FPS

if not cap.isOpened():
    print("Error: No se puede acceder a la cámara.")
    exit()

last_process_time = time.time()
process_interval = 0.5  # Procesar 2 frames por segundo (1/2 = 0.5 segundos)

# Crear ventana una sola vez
cv2.namedWindow('Sign Language Detection', cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el cuadro de la cámara.")
        break
    
    # Redimensionar el frame para procesamiento más rápido
    frame = cv2.resize(frame, (640, 480))
    
    current_time = time.time()
    
    # Procesar frame cada 0.5 segundos (2 FPS para el procesamiento)
    if current_time - last_process_time >= process_interval:
        # Procesar frame con YOLO
        results_yolo = model(frame, conf=0.5)
        
        for result in results_yolo[0].boxes.data:
            x1, y1, x2, y2 = map(int, result[:4])
            conf = float(result[4])
            class_id = int(result[5])
            class_name = results_yolo[0].names[class_id]
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_name} {conf:.2f}', 
                      (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 
                      0.6, (0, 255, 0), 1)
        
        last_process_time = current_time
    
    # Mostrar el frame en tiempo real (30 FPS)
    cv2.imshow('Sign Language Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 608x800 (no detections), 3018.2ms
Speed: 164.2ms preprocess, 3018.2ms inference, 31.1ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1462.8ms
Speed: 7.8ms preprocess, 1462.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1424.6ms
Speed: 10.9ms preprocess, 1424.6ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1602.0ms
Speed: 7.6ms preprocess, 1602.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1625.8ms
Speed: 8.0ms preprocess, 1625.8ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1556.1ms
Speed: 12.7ms preprocess, 1556.1ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1552.1ms
Speed: 8.6ms preprocess, 1552.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 1530.2m

## Sin Media Pipe para el modelo 3

In [6]:
# Cargar el modelo YOLOv8 y optimizar
model = YOLO('best_3.pt')
model.fp16 = True  # Habilitar inferencia en FP16

In [7]:
# Configurar la cámara para menor resolución
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Reducir resolución
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)  # Limitar FPS

if not cap.isOpened():
    print("Error: No se puede acceder a la cámara.")
    exit()

last_process_time = time.time()
process_interval = 0.5  # Procesar 2 frames por segundo (1/2 = 0.5 segundos)

# Crear ventana una sola vez
cv2.namedWindow('Sign Language Detection', cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el cuadro de la cámara.")
        break
    
    # Redimensionar el frame para procesamiento más rápido
    frame = cv2.resize(frame, (640, 480))
    
    current_time = time.time()
    
    # Procesar frame cada 0.5 segundos (2 FPS para el procesamiento)
    if current_time - last_process_time >= process_interval:
        # Procesar frame con YOLO
        results_yolo = model(frame, conf=0.5)
        
        for result in results_yolo[0].boxes.data:
            x1, y1, x2, y2 = map(int, result[:4])
            conf = float(result[4])
            class_id = int(result[5])
            class_name = results_yolo[0].names[class_id]
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_name} {conf:.2f}', 
                      (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 
                      0.6, (0, 255, 0), 1)
        
        last_process_time = current_time
    
    # Mostrar el frame en tiempo real (30 FPS)
    cv2.imshow('Sign Language Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 608x800 (no detections), 562.2ms
Speed: 26.7ms preprocess, 562.2ms inference, 7.6ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 318.1ms
Speed: 10.1ms preprocess, 318.1ms inference, 3.7ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 323.4ms
Speed: 5.4ms preprocess, 323.4ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 307.8ms
Speed: 0.0ms preprocess, 307.8ms inference, 6.3ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 1 K, 291.5ms
Speed: 4.3ms preprocess, 291.5ms inference, 14.8ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 1 K, 261.9ms
Speed: 0.0ms preprocess, 261.9ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 1 K, 283.0ms
Speed: 0.0ms preprocess, 283.0ms inference, 1.9ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 241.6ms
Speed: 0.0ms preprocess, 241.6ms inference, 14.0ms 